In [2]:
import os
os.getcwd()

'/kaggle/working'

In [4]:
!git pull https://github.com/Lolleeee/MIEEG

From https://github.com/Lolleeee/MIEEG
 * branch            HEAD       -> FETCH_HEAD
Already up to date.


In [3]:
!git clone https://github.com/Lolleeee/MIEEG
import os 
os.chdir("MIEEG")

fatal: destination path 'MIEEG' already exists and is not an empty directory.


In [5]:
import numpy as np
matrix = np.array(
    [
        [None, "Fp1", None, "Fp2", None],
        ["F7", "F3", "Fz", "F4", "F8"],
        ["FC5", "FC1", "Cz", "FC2", "FC6"],
        ["T7", "C3", "CP1", "C4", "T8"],
        ["TP9", "CP5", "CP2", "CP6", "TP10"],
        ["P7", "P3", "Pz", "P4", "P8"],
        ["PO9", "O1", "Oz", "O2", "PO10"],
    ]
)



In [ ]:
from packages.models.autoencoder_skip import Conv3DAE 
import torch
model = Conv3DAE(
        use_convnext=True, 
        drop_p=0.1, 
        hidden_dims=[64, 128, 256],
        encoder_skip_connections=[(0, 2)],
        decoder_skip_connections=[(0, 2)],
        latent_dim=512
    )

In [8]:
from packages.train.training import train_model
from packages.train.loss import CustomMSELoss, PerceptualLoss, CustomL1Loss
from packages.io.file_loader import get_data_loaders
import torch
import os

from packages.data_objects.dataset import TorchDataset

dataset_path = "/kaggle/input/64nooverlap05"
# Dummy training loop
optimizer = torch.optim.AdamW

mae = torch.nn.L1Loss
criterion = CustomMSELoss(scale=1, masked=True, matrix=matrix )
config = {
    'weight_decay': 1e-4,
    'epochs': 50,
    'EarlyStopping' : {'patience': 20, 'min_delta': 0.01},
    'BackupManager': {'backup_interval': 10, 'backup_path': './model_backups'},
    'ReduceLROnPlateau': {'mode': 'min', 'patience': 5, 'factor': 0.0},
    'history_plot': {'plot_type': 'extended', 'save_path': './training_history'},
    'grad_clip': 1.0,
    'use_amp': False,
    'grad_logging_interval' : None,
    'asym_lr': None
}

dataset = TorchDataset(root_folder=dataset_path)

train_loader, val_loader, _ = get_data_loaders(dataset, sets_size={'train': 0.7, 'val': 0.3, 'test': 0.}, norm_axes=(0, 4), batch_size = 64)

Calculating global parameters: 100%|██████████| 338/338 [00:15<00:00, 21.14it/s]


In [9]:
model = train_model(model, train_loader=train_loader, val_loader=val_loader, loss_criterion=criterion, optimizer=optimizer, config=config, metrics={})

Epochs:   0%|          | 0/50 [00:00<?, ?it/s]

Training Batches:   0%|          | 0/338 [00:00<?, ?it/s]

[INFO] Train Loss: 0.6232
[INFO] Val Loss: 0.5630
[INFO] Best model saved at epoch 1 with metric 0.5630
[INFO] Updated Learning Rate: 0.001000


Training Batches:   0%|          | 0/338 [00:00<?, ?it/s]

[INFO] Train Loss: 0.5301
[INFO] Val Loss: 0.5234
[INFO] Best model saved at epoch 2 with metric 0.5234


...

[INFO] Train Loss: 0.4243
[INFO] Val Loss: 0.4246
[INFO] Best model saved at epoch 10 with metric 0.4246


Training Batches:   0%|          | 0/338 [00:00<?, ?it/s]

[INFO] Train Loss: 0.4214
[INFO] Val Loss: 0.4242
[INFO] Best model saved at epoch 11 with metric 0.4242


Training Batches:   0%|          | 0/338 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [12]:
from packages.train.testing import autoencoder_test_plots
autoencoder_test_plots(model, val_loader, nsamples = 5)

RuntimeError: Expected weight to be a vector of size equal to the number of channels in input, but got weight of shape [128] and input of shape [128, 7, 5, 64]